In [1]:
import numpy as np
import pandas as pd

In [7]:
df_normal = pd.read_csv("../output/normal/normal.csv")

In [8]:
df_water = pd.read_csv("../output/water_army/water_army.csv")

In [9]:
water_army = df_water["留言內容"].tolist()
water_army

['門市空間很大，沙發擴設很大感試坐起來超舒服～完全可以慢慢挑選，沒有壓力！',
 '這次買沙發，最有感的就是店員的態度很好，不會一直推銷，反而是先了解我們的需求，再推薦適合的款式。我有特別問到家裡有長輩要坐，有沒有支撐性比較好的款式，結果店員立刻介紹了幾款適合的，還分析了它們的差異，讓人覺得很放心。👍',
 '大家這次好市多特價買了些什麼？我家必買這三樣東西：1.港式蘿蔔糕 2.減糖黑十穀 3.好自在衛生棉1.港式蘿蔔糕一定要買這牌的，煎起來又香又軟非常好吃～而且蘿蔔糕上面有紋路，方便切一塊一塊2.減糖黑十穀奶奶愛喝，已購買。雖然一箱裡面有超多包，但奶奶不到半年就喝光光，優惠的時候一定要趕快囤不然奶奶會念XD奶奶表示這包真的很香很好喝一包份量超多！3.好自在衛生棉我有算過好市多的份量雖然比較大包但平均下來便宜很多！大家特價的時候必買什麼呢？！',
 '1. 烤雞必買啦～還有薯餅！2. 想問減糖黑十穀特價到什麼時候？3. （原po回覆：）到3月底4. 1.這款蘿蔔糕我愛！可以當早餐也可以當宵夜5. 減糖黑十穀有推～一包＋燕麥可以直接當午餐6. 我猜大家應該必買衛生紙吧哈哈哈哈7. 北海道特濃燕麥、洗衣精8. 牛奶～可頌～草莓蛋糕～豪華凱薩沙拉9. 2我每次特價都一次抱三箱回家10. 我必買美白牙貼！超划算',
 '自從搬出來住之後，就很喜歡自己弄三餐既有儀式感，吃得又更健康～也很喜歡去全聯採買（有特價最好哈哈哈）像是關廟麵＆超能多穀力就是我日常必買早餐會泡杯芝麻牛奶搭配麵包之類的，來開啟我美好的一天我去年第一次買，直接愛上～真的超級香芝麻控人必收！！！晚餐因為沒有買電鍋，加上我是麵食派基本上就是煮麵＋雞或魚肉忠清北道這個泡菜也是我回購好幾次，很韓式（有時候還有特價）零食我會買不同口味洋芋片來試順便配電影算是生活中的一點小樂趣！以上就是我全聯清單，歡迎大家跟我分享必買清單XD',
 '我也是麵食派，推薦你台南關廟廟，很Q喔芝麻控人報到，這罐也是我必買清單哈哈哈魚肉建議可以去好市多買更划算推推超能多穀力杏仁口味也好喝想問不加牛奶好喝嗎單喝也好喝！！但拍謝我也愛牛奶配電影最對味的還是蚵仔煎😂我都幫我爸媽買耶，沒想到能變芝麻牛奶😍我剛買杏仁口味我必買優格跟冷凍花椰菜米～多穀力也常買，拿來泡豆漿',
 '之前看到有人分享這個好像不錯？因為我本身蠻愛喝燕麥奶的 但沒有嘗試過沖泡

In [10]:
import torch
from tqdm import tqdm
from transformers import BertModel, BertTokenizer

In [11]:
torch.cuda.is_available()

True

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
tokenizer = BertTokenizer.from_pretrained("bert-base-chinese")
model = BertModel.from_pretrained("bert-base-chinese")
model.eval()
model = model.to(device)

In [14]:
records = []

In [15]:
water_army_lte_128, water_army_gt_128_lte_256, water_army_gt_256 = [], [], []
for water in water_army:
    if len(water) <= 128:
        water_army_lte_128.append(water)
    elif len(water) <= 256:
        water_army_gt_128_lte_256.append(water)
    else:
        water_army_gt_256.append(water)

In [16]:
for text in tqdm(water_army_lte_128):
    inputs = tokenizer(
        text, padding="longest", truncation=True, return_tensors="pt", max_length=128
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()

    record = {"text": text}

    for i, val in enumerate(cls_embedding):
        record[f"v{i}"] = val

    records.append(record)

  0%|          | 0/531 [00:00<?, ?it/s]C:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
100%|██████████| 531/531 [00:04<00:00, 111.98it/s]


In [17]:
for text in tqdm(water_army_gt_128_lte_256):
    inputs = tokenizer(
        text, padding="longest", truncation=True, return_tensors="pt", max_length=256
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()

    record = {"text": text}

    for i, val in enumerate(cls_embedding):
        record[f"v{i}"] = val

    records.append(record)

100%|██████████| 106/106 [00:01<00:00, 59.66it/s]


In [18]:
for text in tqdm(water_army_gt_256):
    inputs = tokenizer(
        text,
        padding="longest",
        truncation=True,
        return_tensors="pt",
        max_length=512,
        stride=128,
        return_overflowing_tokens=True,
        return_attention_mask=True,
    )
    inputs = {
        k: v.to(device)
        for k, v in inputs.items()
        if k in ["input_ids", "attention_mask"]
    }

    cls_embeddings = []

    for i in range(inputs["input_ids"].shape[0]):
        input_ids = inputs["input_ids"][i].unsqueeze(0)
        attention_mask = inputs["attention_mask"][i].unsqueeze(0)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            cls_vec = outputs.last_hidden_state[:, 0, :]
            cls_embeddings.append(cls_vec.squeeze().cpu().numpy())

    final_embedding = np.mean(cls_embeddings, axis=0)

    record = {"text": text}
    for i, val in enumerate(final_embedding):
        record[f"v{i}"] = val
    records.append(record)

100%|██████████| 110/110 [00:03<00:00, 33.42it/s]


In [19]:
df_water = pd.DataFrame(records)
df_water.to_parquet("../output/water_army/water_army.parquet", engine="pyarrow")

In [20]:
normal = df_normal["留言內容"].tolist()
normal

['一個人吃飯 被安排到只差一個位子垃圾桶位置 都快吃完 連續被2次請求 更親近垃圾桶 再等5分鐘再拼翻桌率 不行嗎？ 坐那個位置 心情都低落 體諒人客多。 當初就拒絕單人用餐就好',
 '東西算好吃，口味不錯，可能生意太好，可惜服務態度不是很好',
 '師父炒得很隨便 跟朋友用餐 朋友的菜很鹹 不然就沒味道 …',
 '還可以更好。',
 '每來必吃',
 '還可以',
 '師傅炒菜品質差異太大 每個禮拜都來吃 但最近遇到不好吃的頻率有點太高 …',
 '很好吃！出餐動作很快價格合理',
 '好吃。雙人套餐挺豐盛。酸梅湯好喝',
 '不曉得是點牛排還是點橡皮筋，每一塊都完全咬不爛，咬一咬只能吐出來，一塊都吞不了，找人來問問推說供應商的問題，說今天的牛肉都這樣。有問題的東西還能賣喔，錢還照算，此生吃過最火大的店就這家了，趕快關門洗洗睡吧。',
 '高麗菜都是梗還燒焦，飯是硬的：）',
 '吃得有夠飽，但味道偏重一點，適合重口味的人來。',
 '感覺以前比較好吃',
 '2020/06/20， 三人套餐 肋眼牛排， 肉質很棒， 搭配蒜酥是本餐最好吃 蜜汁雞腿， 甜甜的醬其實有點怪，下次改乾煎試試 …',
 '餐點ok，只是拍照打卡送起司蛋有要求寫”好吃”之類的字眼有點意外',
 '近期吃過最好吃的平價鐵板燒',
 '味道好極了！會回來的😎 …',
 '好吃好吃好吃👍👍👍👍👍 …',
 '出餐速度快 好吃',
 '真的一顆星也不想給.外帶的飯是給人吃的嗎.白飯跟粥一樣爛.怎麼能煮成這樣.肉不夠鹹. 希望你們可以做好好的東西給別人吃好不好.',
 '微辣炒出來的變大辣⋯傻眼，高麗菜超硬，請問是剛摘起來的嗎？？食材跟一般大埔沒兩樣也敢賣那麼貴，我有兩百組帳號我一定按兩百個一星給你',
 '牛肉 羊肉都還可以 酸梅汁很好喝 就是價格有點偏貴',
 '平價好吃的鐵板燒，推薦一定點他們的點心 蛋糕。',
 '以前常吃，但今天回訪感覺品質不若以往 牛肉肉質普通，菜炒得過油，九層塔蛋吃得出結塊的鹽 還被師傅煎的虱目魚噴出的油燙到眼皮 …',
 '櫃檯阿姨很沒禮貌，服務態度超級差',
 '從以前就吃過，後來她們搬家了還是喜歡去吃，口味我很喜歡',
 '請加強員工訓練 站櫃台那位 給人感受十分不悅',
 '相當平價的鐵板料理，份量都夠享用，自助式的味噌湯與白飯，還有解膩的檸檬水與烏梅汁，

In [21]:
normal_lte_128, normal_gt_128_lte_256, normal_gt_256 = [], [], []
for norm in normal:
    if len(norm) <= 128:
        normal_lte_128.append(norm)
    elif len(norm) <= 256:
        normal_gt_128_lte_256.append(norm)
    else:
        normal_gt_256.append(norm)

In [22]:
normal_token = []

In [23]:
for text in tqdm(normal_lte_128):
    inputs = tokenizer(
        text, padding="longest", truncation=True, return_tensors="pt", max_length=128
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()

    record = {"text": text}

    for i, val in enumerate(cls_embedding):
        record[f"v{i}"] = val

    normal_token.append(record)

100%|██████████| 1731/1731 [00:14<00:00, 120.83it/s]


In [24]:
for text in tqdm(normal_gt_128_lte_256):
    inputs = tokenizer(
        text, padding="longest", truncation=True, return_tensors="pt", max_length=256
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()

    record = {"text": text}

    for i, val in enumerate(cls_embedding):
        record[f"v{i}"] = val

    normal_token.append(record)

100%|██████████| 175/175 [00:02<00:00, 62.51it/s]


In [25]:
for text in tqdm(normal_gt_256):
    inputs = tokenizer(
        text,
        padding="longest",
        truncation=True,
        return_tensors="pt",
        max_length=512,
        stride=128,
        return_overflowing_tokens=True,
        return_attention_mask=True,
    )
    inputs = {
        k: v.to(device)
        for k, v in inputs.items()
        if k in ["input_ids", "attention_mask"]
    }

    cls_embeddings = []

    for i in range(inputs["input_ids"].shape[0]):
        input_ids = inputs["input_ids"][i].unsqueeze(0)
        attention_mask = inputs["attention_mask"][i].unsqueeze(0)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            cls_vec = outputs.last_hidden_state[:, 0, :]
            cls_embeddings.append(cls_vec.squeeze().cpu().numpy())

    final_embedding = np.mean(cls_embeddings, axis=0)

    record = {"text": text}
    for i, val in enumerate(final_embedding):
        record[f"v{i}"] = val
    normal_token.append(record)

100%|██████████| 144/144 [00:03<00:00, 40.05it/s]


In [26]:
df_norm = pd.DataFrame(normal_token)
df_norm.to_parquet("../output/normal/normal_comment.parquet", engine="pyarrow")

In [24]:
df = pd.read_csv("../output/test/raw_data.csv")
df

,nm_name,st_name,nm_city,st_address,user_name,user_id,is_local_guide,review_count,photo_count,rating_star,time_num,time_unit,months_ago,content_clean,create_date,update_date,water_army
0,南機場,玉米家,台北市, 100台北市中正區中華路二段315巷5弄,土司（栗悟飯和龜派氣功）,116133179077073984081,TRUE,81,290,2,3,年,36.1,----- 5/11回覆店家留言 第一印象我並沒有多差，當然就是有上過必比登當然吃看看。我沒...,2025-05-01,2025-05-01,F
1,南機場,玉米家,台北市, 100台北市中正區中華路二段315巷5弄,Chou Juliet,110070635888045386698,TRUE,949,2,5,1,年,12.1,《烤玉米100*2》 沒有排隊人潮，價格明確。 選好玉米及醬料後，可拿號碼牌並掃描QRcod...,2025-05-01,2025-05-01,F
2,南機場,南機場無名蛋餅,台北市, 100台北市中正區中華路二段309巷24號,Rose B,107924492121971214904,TRUE,312,1,5,5,年,60.1,「下雨天不出攤」 無名蛋餅配蘿蔔乾、韭菜盒子🥙加蛋🥚 …,2025-05-01,2025-05-01,F
3,南機場,"山內雞肉 南機場夜市店|雞肉飯、雞腿飯,帶皮腱肉飯、咖哩飯、雞湯拉麵、便當外送",台北市, 100台北市中正區中華路二段307巷20-3號,Yuan Hung Cheng,111531120177861197602,TRUE,442,6,4,6,個月,0.1,「山內雞肉飯」位於南機場夜市，是一間以家常風味聞名的小吃攤，擁有一系列簡單卻令人難忘的餐點。...,2025-05-01,2025-05-01,T
4,南機場,玉米家,台北市, 100台北市中正區中華路二段315巷5弄,nien,114251924883120571161,TRUE,576,3,2,2,年,24.1,「玉米家」位於南機場夜市裡面的攤販之一，老闆娘服務態度很好，排隊人潮很多，價格NT60-10...,2025-05-01,2025-05-01,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
604,士林夜市,四季麵疙瘩,台北市,士林區福德路13號,Debbie Rula,103522117758446210900,TRUE,47,75,5,6,年,72,很棒的台灣街頭小吃,2025-05-09,2025-05-09,F
605,士林夜市,喫尤平價鐵板燒 - 士林店,台北市,士林區大南路50號,Triệu Nguyễn Phương Đài (Joyce),103018645360655431404,FALSE,4,2,5,10,個月,10,白飯跟湯都吃到飽，又便宜又好吃，我已經來好多次了，幹前雞腿跟牛排是最好吃的，再加90塊就有一...,2025-05-09,2025-05-09,F
606,士林夜市,ajussi-阿啾西拉麵圖書館-士林旗艦店,台北市,士林區基河路165號,游哲睿,103374740874646914580,TRUE,58,21,5,1,年,12,新奇又美味,2025-05-09,2025-05-09,F
607,士林夜市,海友十全排骨,台北市,士林區大東路49號,林品瑩,108892501482035367782,FALSE,1,1,5,7,年,84,NaN,2025-05-08,2025-05-08,F


In [25]:
comments = df["content_clean"].tolist()
comments

['----- 5/11回覆店家留言 第一印象我並沒有多差，當然就是有上過必比登當然吃看看。我沒事找事做嗎？ 我點完在旁邊等，我才看到一堆人用手去戳去捏 阿當這裡水果店？我點了是能反悔說我不要嗎？ 你開火鍋店關我什麼事情，就事論事 我第一次消費，我感受到的就是「不衛生」 如果您覺得我這是無理取鬧，找麻煩，隨意您想 反正我我絕對不可能再去第二次，您可以不用再繼續回覆我。 畢竟我說的都是當天親眼看到的事情。 如果是注重衛生、防範疫情的客人 拜託不要去買了，你只會一肚子火 ----------- 舊留言 看到米其林推薦，今天買一次試試看，有點失望。 第一點：挑玉米居然讓客人徒手捏 這個真的讓我有點覺得噁心，尤其現在疫情當前 旁邊沒有放任何消毒或是拋棄式手套讓客人使用。 我看好幾個客人（感覺熟客），直接徒手捏 還有拿起來整隻握住，挑完發現不對又丟回去繼續捏，窩操我超後悔點了一支。 第二點：口味鹹 可能太忙可能我不習慣，其他人說能吃到玉米的甜味，今天我吃不到玉米的甜味而且還超鹹。 玉米口感很ok,口味我真的無法...鹹到我直接去買飲料 以上，可能不會買的第二次了。那個徒手捏玉米我真的..不行，不管你有沒有後續烤過高溫消毒，我都覺得很噁',
 '《烤玉米100*2》 沒有排隊人潮，價格明確。 選好玉米及醬料後，可拿號碼牌並掃描QRcode知道自己該何時回來取玉米，不用一直耗在那邊。 玉米溫度不高，有點小失望，但調味不錯，醬料也可以自行選擇或雙拼。 整體而言還不錯。',
 '「下雨天不出攤」 無名蛋餅配蘿蔔乾、韭菜盒子🥙加蛋🥚 …',
 '「山內雞肉飯」位於南機場夜市，是一間以家常風味聞名的小吃攤，擁有一系列簡單卻令人難忘的餐點。店內的雞肉飯是主打之一，米飯粒粒分明，雞肉細嫩多汁，搭配上店家特製的雞油與醬汁，讓每一口都充滿香氣。雞肉飯的口感輕盈，不過於油膩，是一道經典且親民的台灣美食。 另一特色是帶皮腱肉飯，這款飯以帶皮的豬腱肉為主角，腱肉煮得非常入味，肉質鮮嫩卻不失彈性，帶皮部分富有膠質，吃起來更添豐富口感，與白飯一同享用相當滿足。 雞腿飯則是更豪華的選擇，整隻雞腿煮得恰到好處，肉質鮮嫩且保有多汁的口感，搭配同樣優質的醬油汁與白飯，每一口都令人回味無窮。 這家店的特色辣椒醬油是靈魂所在，微辣的醬油完美提升了每道菜的風味，辣度適中，帶有濃郁的蒜香與辛香味，無論淋在雞肉飯還是腱肉飯

In [ ]:
comments_lte_128, comments_gt_128_lte_256, comments_gt_256 = [], [], []
for comment in comments:
    if pd.isna(comment):
        pass
    elif len(comment) <= 128:
        comments_lte_128.append(comment)
    elif len(comment) <= 256:
        comments_gt_128_lte_256.append(comment)
    else:
        comments_gt_256.append(comment)

In [ ]:
batch_size = 10000
for i in range(0, len(comments_lte_128), batch_size):
    batch_end = min(i + batch_size, len(comments_lte_128))
    batch = comments_lte_128[i:batch_end]
    batch_records = []

    for text in tqdm(batch, desc=f"Batch {i // batch_size + 1}"):
        try:
            inputs = tokenizer(
                text,
                padding="max_length",
                truncation=True,
                return_tensors="pt",
                max_length=128,
            )
            inputs = {k: v.to(device) for k, v in inputs.items()}

            with torch.no_grad():
                outputs = model(**inputs)
                cls_embedding = outputs.last_hidden_state[0, 0, :].cpu().numpy()

            record = {"text": text}
            for j, val in enumerate(cls_embedding):
                record[f"v{j}"] = val
            batch_records.append(record)

        except Exception as e:
            print(f"Error on text: {text[:30]} — {e}")
            continue

    df = pd.DataFrame(batch_records)
    df.to_parquet(f"../output/test/comment_lte128_part_{i // batch_size + 1}.parquet")

    del batch_records
    torch.cuda.empty_cache()

Batch 1:   0%|          | 0/518 [00:00<?, ?it/s]

Batch 1: 100%|██████████| 518/518 [00:05<00:00, 86.72it/s]


In [ ]:
batch_size = 10000
for i in range(0, len(comments_gt_128_lte_256), batch_size):
    batch_end = min(i + batch_size, len(comments_gt_128_lte_256))
    batch = comments_gt_128_lte_256[i:batch_end]
    batch_records = []

    for text in tqdm(batch, desc=f"Batch {i // batch_size + 1}"):
        try:
            inputs = tokenizer(
                text,
                padding="max_length",
                truncation=True,
                return_tensors="pt",
                max_length=256,
            )
            inputs = {k: v.to(device) for k, v in inputs.items()}

            with torch.no_grad():
                outputs = model(**inputs)
                cls_embedding = outputs.last_hidden_state[0, 0, :].cpu().numpy()

            record = {"text": text}
            for j, val in enumerate(cls_embedding):
                record[f"v{j}"] = val
            batch_records.append(record)

        except Exception as e:
            print(f"Error on text: {text[:30]} — {e}")
            continue

    df = pd.DataFrame(batch_records)
    df.to_parquet(
        f"../output/test/comment_gt128_lte256_part_{i // batch_size + 1}.parquet"
    )

    del batch_records
    torch.cuda.empty_cache()

Batch 1: 100%|██████████| 45/45 [00:01<00:00, 41.45it/s]


In [ ]:
batch_size = 10000
for i in range(0, len(comments_gt_256), batch_size):
    batch_end = min(i + batch_size, len(comments_gt_256))
    batch = comments_gt_256[i:batch_end]
    batch_records = []

    for text in tqdm(batch, desc=f"Batch {i // batch_size + 1}"):
        try:
            inputs = tokenizer(
                text,
                padding="max_length",
                truncation=True,
                return_tensors="pt",
                max_length=512,
                stride=128,
                return_overflowing_tokens=True,
                return_attention_mask=True,
            )
            inputs = {
                k: v.to(device)
                for k, v in inputs.items()
                if k in ["input_ids", "attention_mask"]
            }

            cls_embeddings = []
            for chunk_idx in range(inputs["input_ids"].shape[0]):
                input_ids = inputs["input_ids"][chunk_idx].unsqueeze(0)
                attention_mask = inputs["attention_mask"][chunk_idx].unsqueeze(0)

                with torch.no_grad():
                    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                    cls_vec = outputs.last_hidden_state[:, 0, :]
                    cls_embeddings.append(cls_vec.squeeze().cpu().numpy())

            final_embedding = np.mean(cls_embeddings, axis=0)

            record = {"text": text}
            for j, val in enumerate(final_embedding):
                record[f"v{j}"] = val

            batch_records.append(record)

        except Exception as e:
            print(f"Error on text: {text[:30]}... => {e}")
            continue

    df = pd.DataFrame(batch_records)
    df.to_parquet(f"../output/test/comment_gte256_part_{i // batch_size + 1}.parquet")

    del batch_records
    torch.cuda.empty_cache()

Batch 1: 100%|██████████| 16/16 [00:00<00:00, 21.36it/s]
